In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from pykospacing import Spacing

import torch
import spacy
from transformers import ElectraModel, ElectraTokenizer

nlp = spacy.load("ko_core_news_sm")  # 한국어 모델 로드
tqdm.pandas()                        # tqdm과 pandas 통합

In [6]:
# 불용어 파일 로드
stopwords_file = '../../../../../data/stopwords-ko.txt'
with open(stopwords_file, 'r', encoding='utf-8') as f:
    stopwords = set(line.strip() for line in f if line.strip())

# 불용어 처리 함수 정의
def remove_stopwords(text, stopwords):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if token.text not in stopwords and not token.is_punct and not token.is_space]
    return filtered_tokens


spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 텍스트의 임베딩을 얻는 함수
def get_koelectra_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

In [ ]:
df = pd.read_excel('../../../../../data/filtered_news.xlsx')

# 데이터 전처리 및 토큰화
df = df[:100]
df['cleaned'] = df['feature'].apply(lambda x: remove_stopwords(preprocessing(x), stopwords))

In [9]:
# KoELECTRA 토크나이저와 모델을 로드합니다.
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')


# 각 처리된 피처에 대해 함수를 적용합니다..
df['cleaned_text'] = df['cleaned'].apply(lambda x: ' '.join(x))
df['koelectra_embedding'] = df['cleaned_text'].apply(get_koelectra_embeddings)


# 시각화를 위해 DataFrame으로 변환합니다.
koelectra_embeddings_df = pd.DataFrame(df['koelectra_embedding'].tolist())